In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import requests
import speech_recognition as sr
from io import BytesIO
from matplotlib import patches
from os import path
from PIL import Image
from pprint import pprint
from IPython.display import HTML

In [3]:
class Speech:
    def emotions_from_image(self, image_path):
        headers = {
            'Ocp-Apim-Subscription-Key': self.kEMOTION_KEY,
            'Content-type': 'application/octet-stream'
        }
        params = {
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'false',
            'returnFaceAttributes': 'headPose,emotion',
        }
        with open(image_path, 'rb') as f:
            img_data = f.read()
        response = requests.post(self.kEMOTION_URL, params=params, headers=headers, data=img_data)
        faces = response.json()
        fr = faces[0]["faceRectangle"]
        fa = faces[0]["faceAttributes"]
        headPose = fa["headPose"]
        emotions = fa["emotion"]
        emotion = max(emotions, key=lambda key: emotions[key])
        return emotion, headPose
    
    
    def speech_to_text(self, file_path):
        audio_file = path.join(path.dirname(path.realpath('__file__')), file_path)
        r = sr.Recognizer()
        with sr.AudioFile(file_path) as source:
            audio = r.record(source)
        try:
            raw_text = r.recognize_bing(audio, key=self.kSPEECH_TO_TEXT_KEY)
            document = {'documents' : [
              {'id': '1', 'language': 'en', 'text': raw_text}
            ]}
            return document
        except sr.UnknownValueError:
            return("Microsoft Bing Voice Recognition could not understand audio")
        except sr.RequestError as e:
            return("Could not request results from Microsoft Bing Voice Recognition service; {0}".format(e))


    def extract_sentiment(self, text):
        headers   = {"Ocp-Apim-Subscription-Key": self.kTEXT_ANALYSIS_KEY}
        response  = requests.post(self.kSENTIMENT_URL, headers=headers, json=text)
        sentiments = response.json()
        return sentiments['documents'][0]['score']


    def extract_keywords(self, text):
        headers   = {"Ocp-Apim-Subscription-Key": self.kTEXT_ANALYSIS_KEY}
        response  = requests.post(self.kKEY_PHRASE_URL, headers=headers, json=text)
        key_phrases = response.json()
        return key_phrases['documents'][0]['keyPhrases']


    def get_search_results(self, search_term):
        headers = {"Ocp-Apim-Subscription-Key" : self.kSEARCH_KEY}
        params  = {"q": search_term, "textDecorations":True, "textFormat":"HTML"}
        response = requests.get(self.kSEARCH_URL, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        rows = "\n".join(["""<tr>
                           <td><a href=\"{0}\">{1}</a></td>
                           <td>{2}</td>
                         </tr>""".format(v["url"],v["name"],v["snippet"]) for v in search_results["webPages"]["value"]])
        return HTML("<table>{0}</table>".format(rows))

    
    def __init__(self):
        # Face API
        self.kEMOTION_KEY = "34162b79f80a44f58c6629a083929c15"
        self.kEMOTION_URL = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect'
        # Speech to Text API
        self.kSPEECH_TO_TEXT_KEY = "2ed41467970e4791bec9a48388bb416e"
        # Text Analysis API
        self.kTEXT_ANALYSIS_KEY = "e552bd5890934ed595dbf8d7fd05816f"
        text_analytics_base_url = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/"
        self.kSENTIMENT_URL = text_analytics_base_url + "sentiment"
        self.kKEY_PHRASE_URL = text_analytics_base_url + "keyPhrases"
        # Bing Web API
        self.kSEARCH_KEY = "9db3ddee989743f48b738680d30c5bab"
        self.kSEARCH_URL = "https://api.cognitive.microsoft.com/bing/v7.0/search/"

In [4]:
speech = Speech()
emotion, headPose = speech.emotions_from_image("main002.jpg")
print(emotion)
print(headPose)

happiness
{'pitch': 0.0, 'roll': -0.2, 'yaw': -0.8}


In [5]:
text = speech.speech_to_text(file_path='sample.wav')
print(text)

{'documents': [{'id': '1', 'language': 'en', 'text': 'Every word and phrase he speaks is true he put his last cartridge into the gun and fired they took their kids from the public school drive the screws straight into the would keep the hatch tight in the watch constant several that wine with a quick snip of the knife paper will dry out when wet slide the catch back and open the desk help the week to preserve their strength they selling smile get you friends.'}]}


In [6]:
sentiment = speech.extract_sentiment(text)
print(sentiment)

0.8571277856826782


In [7]:
keywords = speech.extract_keywords(text)
print(keywords)

['watch constant', 'knife paper', 'quick snip', 'wine', 'wet slide', 'strength', 'kids', 'public school', 'catch', 'hatch tight', 'word', 'phrase', 'smile', 'friends', 'cartridge', 'week']


In [8]:
search_results = speech.get_search_results(keywords[3])
search_results

Wine.com - Official Site,Shop the world's largest wine store with thousands of domestic and imported wines delivered right to your door. Chat online with wine experts 7 days a week.
Wine - Wikipedia,"Wine (from Latin vinum) is an alcoholic beverage made from grapes, generally Vitis vinifera, fermented without the addition of sugars, acids, enzymes, water, or other ..."
Wine - Official Site,"Wine (originally an acronym for ""Wine Is Not an Emulator"") is a compatibility layer capable of running Windows applications on several POSIX-compliant operating ..."
Total Wine & More - Official Site,"Over 8,000 wines, 3,000 spirits & 2,500 beers with the best prices, selection and service at America’s Wine Superstore. Shop online for delivery or in-store pick up."
"Wine Recipes, Guides, Bars & Ideas | Food & Wine","Looking for the best wines? Find the best wine recommendations, ideas and news on Food & Wine."
Wine Spectator - Official Site,"350,000+ expert wine ratings, with full reviews, including tasting notes, score, price and when to drink. Learn more, drink better: The essentials of wine, vintage ..."
Wines.com,Wine Tasting Forums. Join the conversation on the WineBoard. It’s one of the richest sources of tasting notes and wine information on the web.
Gourmet Deals | Woot,"Wine Line Random Accessory 15-Pack. Either get on the wine line or get in the wine line, depending on where you're from."
Wine (software) - Wikipedia,Wine (recursive backronym for Wine Is Not an Emulator) is a free and open-source compatibility layer that aims to allow computer programs (application software and ...
Wine | HuffPost,Wine news and opinion ... Tap here to turn on desktop notifications to get the news sent straight to you.
